# Generating a set of Total Field anomaly data for a model

Notebook to open a dictionary with the Total Field Anomaly data for a set of geometrical objects.

#### Import libraries

In [ ]:
%matplotlib inline
from IPython.display import Markdown as md
from IPython.display import display as dp
import string as st
import sys
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import datetime

from fatiando.utils import ang2vec, vec2ang
from fatiando.mesher import Sphere, Prism,PolygonalPrism
from fatiando.gravmag import sphere,prism, polyprism

In [ ]:
notebook_name = 'synthetic_data.ipynb'

#### Black background figures

In [ ]:
plt.style.use('classic')

#### Importing auxiliary functions

In [ ]:
dir_modules = '../../mypackage'
sys.path.append(dir_modules)

In [ ]:
import auxiliary_functions as func

#### Loading properties of a set of geometrical objects

In [ ]:
with open('data/model_sphere_remanent.pickle') as f:
        model_sphere_remanent = pickle.load(f)

In [ ]:
with open('data/model_sphere_induced.pickle') as f:
        model_sphere_induced = pickle.load(f)

#### Loading the grid parameters

In [ ]:
with open('data/regular_grid.pickle') as f:
        regular = pickle.load(f)

In [ ]:
with open('data/airborne_survey.pickle') as f:
        airborne = pickle.load(f)

#### Constructing a dictionary

In [ ]:
data_set = dict()

#### List of saved files

In [ ]:
saved_files = []

## Properties of the model

### Dimension and coordinate of the center

In [ ]:
print 'Properties Sphere:\n xc = %.1f m \n yc = %.1f m \n zc = %.1f m \n radius = %.1f m ' %  (model_sphere_remanent['xc'],
                                                                                               model_sphere_remanent['yc'],
                                                                                               model_sphere_remanent['zc'],
                                                                                               model_sphere_remanent['radius'])

### Main field

In [ ]:
inc_gf,dec_gf = model_sphere_remanent['main_field']

In [ ]:
print'Main field inclination: %.1f degree' % inc_gf
print'Main field declination: %.1f degree' % dec_gf

### Magnetization direction

#### Direction w/ the presence of remanent magnetization

In [ ]:
print 'Intensity: %.1f A/m' % model_sphere_remanent['m_R']
print 'Inclination: %.1f degree' % model_sphere_remanent['inc_R']
print 'Declination: %.1f degree' % model_sphere_remanent['dec_R']

In [ ]:
inc_R,dec_R = model_sphere_remanent['inc_R'],model_sphere_remanent['dec_R']

#### Direction w/ only induced magnetization

In [ ]:
print 'Intensity: %.1f A/m' % model_sphere_induced['m_I']
print 'Inclination: %.1f degree' % model_sphere_induced['inc_I']
print 'Declination: %.1f degree' % model_sphere_induced['dec_I']

In [ ]:
inc_I,dec_I = model_sphere_induced['inc_I'],model_sphere_induced['dec_I']

## Calculating the data for Regular grid

### For regular grid

#### Observation area

In [ ]:
print 'Area limits: \n x_max = %.1f m \n x_min = %.1f m \n y_max = %.1f m \n y_min = %.1f m' % (regular['area'][1], regular['area'][0],
                                                                                                regular['area'][3],regular['area'][2])

#### Grid information

In [ ]:
print 'Shape : (%.0f,%.0f)'% regular['shape'] 
print 'Number of data: %.1f' % regular['N']
print 'dx: %.1f m' % regular['dx']
print 'dy: %.1f m ' % regular['dy']
print 'Height: %.1f m' % regular['z_obs']

#### Calculation the data

In [ ]:
data_set['tfa_sphere_RM_reg'] = sphere.tf(regular['x'],regular['y'],regular['z'],
                                          model_sphere_remanent['model'],
                                          inc_gf,dec_gf)

In [ ]:
data_set['tfa_sphere_IM_reg'] = sphere.tf(regular['x'],regular['y'],regular['z'],
                                          model_sphere_induced['model'],
                                          inc_gf,dec_gf)

##### Generating noise for the data set w/ remanet magnetization presence

In [ ]:
np.random.seed(seed=40)

In [ ]:
std_noise = 10.
r = np.random.normal(0.0,std_noise, regular['Nx']*regular['Ny'])
data_set['tfa_obs_sphere_RM_reg'] =  data_set['tfa_sphere_RM_reg'] + r

##### Generating noise for the induced data set

In [ ]:
np.random.seed(seed=40)

In [ ]:
std_noise = 10.
r = np.random.normal(0.0,std_noise, regular['Nx']*regular['Ny'])
data_set['tfa_obs_sphere_IM_reg'] =  data_set['tfa_sphere_IM_reg'] + r

#### Visualization of Total Field Anomaly for regular grid w/ the presence of Remanent magnetization in a  sphere

In [ ]:
title_font = 20
bottom_font = 16
saturation_factor = 1.
plt.close('all')
plt.figure(figsize=(8,8), tight_layout=True)

ranges = saturation_factor*np.abs([data_set['tfa_obs_sphere_RM_reg'].max(), 
                                   data_set['tfa_obs_sphere_RM_reg'].min()]).max()

plt.contourf(regular['y'].reshape(regular['shape']), 
             regular['x'].reshape(regular['shape']),
             data_set['tfa_obs_sphere_RM_reg'].reshape(regular['shape']), 
             20, cmap='RdBu_r',vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('nT')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.title('TFA (RM_reg_grid)', fontsize=title_font)
plt.text(1500.,4500.,'Inc = %.1f | Dec = %.1f'% (inc_R,dec_R), 
         family='ubuntu',style='normal')
plt.text(800.,4100.,'Inc_mf = %.1f | Dec_mf = %.1f'% (inc_gf,dec_gf), 
         family='ubuntu',style='normal' )

file_name = 'figs/regular/noisy_data_tfa_sphere_RM_regular'
plt.savefig(file_name+'.png',dpi=200)
saved_files.append(file_name+'.png')

plt.show()

#### Visualization of Total Field Anomaly for regular grid w/ Induced magnetization sphere

In [ ]:
title_font = 20
bottom_font = 16
saturation_factor = 1.
plt.close('all')
plt.figure(figsize=(8,8), tight_layout=True)

ranges = saturation_factor*np.abs([data_set['tfa_obs_sphere_IM_reg'].max(),
                                   data_set['tfa_obs_sphere_IM_reg'].min()]).max()

plt.contourf(regular['y'].reshape(regular['shape']), 
             regular['x'].reshape(regular['shape']), 
             data_set['tfa_obs_sphere_IM_reg'].reshape(regular['shape']), 
             20, cmap='RdBu_r',vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('nT')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.title('TFA (IM_reg_grid)', fontsize=title_font)
plt.text(1500.,4500.,'Inc = %.1f | Dec = %.1f'% (inc_I,dec_I), 
         family='ubuntu',style='normal')
plt.text(800.,4100.,'Inc_mf = %.1f | Dec_mf = %.1f'% (inc_gf,dec_gf), 
         family='ubuntu',style='normal' )

file_name = 'figs/regular/noisy_data_tfa_sphere_IM_regular'
plt.savefig(file_name+'.png',dpi=200)
saved_files.append(file_name+'.png')

plt.show()

### For Airborne survey

#### Observation area

In [ ]:
print 'Area limits: \n x_max = %.1f m \n x_min = %.1f m \n y_max = %.1f m \n y_min = %.1f m' % (airborne['area'][1],airborne['area'][0],
                                                                                                airborne['area'][3],airborne['area'][2])

#### Airborne survey information

In [ ]:
print 'Shape : (%.0f,%.0f)'% airborne['shape'] 
print 'Number of data: %.1f' % airborne['N']
print 'dx: %.1f m' % airborne['dx']
print 'dy: %.1f m ' % airborne['dy']
print 'Height: %.1f m' % airborne['z_obs']

#### Calculating the data

In [ ]:
data_set['tfa_sphere_RM_airb'] = sphere.tf(airborne['x'],airborne['y'],airborne['z'],
                                          model_sphere_remanent['model'],
                                          inc_gf,dec_gf)

In [ ]:
data_set['tfa_sphere_IM_airb'] = sphere.tf(airborne['x'],airborne['y'],airborne['z'],
                                          model_sphere_induced['model'],
                                          inc_gf,dec_gf)

##### Generating noise for the data w/ the presence of remanence 

In [ ]:
np.random.seed(seed=40)

In [ ]:
std_noise = 10.
r = np.random.normal(0.0,std_noise, airborne['Nx']*airborne['Ny'])
data_set['tfa_obs_sphere_RM_airb'] =  data_set['tfa_sphere_RM_airb'] + r

##### Generating noise for induced magnetization 

In [ ]:
std_noise = 10.
r = np.random.normal(0.0,std_noise, airborne['Nx']*airborne['Ny'])
data_set['tfa_obs_sphere_IM_airb'] =  data_set['tfa_sphere_IM_airb'] + r

#### Visualization of Total Field Anomaly for airborne survey w/ the presence of Remanent magnetization in a  sphere

In [ ]:
title_font = 20
bottom_font = 16
saturation_factor = 1.
plt.close('all')
plt.figure(figsize=(8,8), tight_layout=True)

ranges = saturation_factor*np.abs([data_set['tfa_obs_sphere_RM_airb'].max(), 
                                   data_set['tfa_obs_sphere_RM_airb'].min()]).max()

plt.contourf(airborne['y'].reshape(airborne['shape']), 
             airborne['x'].reshape(airborne['shape']),
             data_set['tfa_obs_sphere_RM_airb'].reshape(airborne['shape']), 
             20, cmap='RdBu_r',vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('nT')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.title('TFA (RM_airborne)', fontsize=title_font)
plt.text(1500.,4500.,'Inc = %.1f | Dec = %.1f'% (inc_R,dec_R), 
         family='ubuntu',style='normal')
plt.text(800.,4100.,'Inc_mf = %.1f | Dec_mf = %.1f'% (inc_gf,dec_gf), 
         family='ubuntu',style='normal' )


file_name = 'figs/airborne/noisy_data_tfa_sphere_RM_airborne'
plt.savefig(file_name+'.png',dpi=200)
saved_files.append(file_name+'.png')


plt.show()

#### Visualization of Total Field Anomaly for regular grid w/ Induced magnetization sphere

In [ ]:
title_font = 20
bottom_font = 16
saturation_factor = 1.
plt.close('all')
plt.figure(figsize=(8,8), tight_layout=True)

ranges = saturation_factor*np.abs([data_set['tfa_obs_sphere_IM_airb'].max(),
                                   data_set['tfa_obs_sphere_IM_airb'].min()]).max()

plt.contourf(airborne['y'].reshape(airborne['shape']), 
             airborne['x'].reshape(airborne['shape']), 
             data_set['tfa_obs_sphere_IM_airb'].reshape(airborne['shape']), 
             20, cmap='RdBu_r',vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('nT')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.title('TFA (IM_airborne)', fontsize=title_font)
plt.text(1500.,4500.,'Inc = %.1f | Dec = %.1f'% (inc_I,dec_I), 
         family='ubuntu',style='normal')
plt.text(800.,4100.,'Inc_mf = %.1f | Dec_mf = %.1f'% (inc_gf,dec_gf), 
         family='ubuntu',style='normal' )


file_name = 'figs/airborne/noisy_data_tfa_sphere_IM_airborne'

plt.savefig(file_name+'.png',dpi=200)

saved_files.append(file_name+'.png')


plt.show()

#### Generating .pickle file

In [ ]:
now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
data_set['metadata'] = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)

In [ ]:
file_name = 'data/data_set.pickle'
with open(file_name, 'w') as f:
    pickle.dump(data_set, f)
    
saved_files.append(file_name)


## Saved files

In [ ]:
with open('reports/report_%s.md' % notebook_name[:st.index(notebook_name, '.')], 'w') as q:
    q.write('# Saved files \n')
    now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
    header = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)
    q.write('\n\n'+header+'\n\n')
    for i, sf in enumerate(saved_files):
        print '%d  %s' % (i+1,sf)
        q.write('*  `%s` \n' % (sf))